Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Dig deeper into kernels by analyzing it with Nsight Compute

In the previous section, we learned to optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenACC. Moreover, we used NVIDIA Nsight Systems to get a system-wide performance analysis. Now, let's dig deeper and profile the kernel with the Nsight Compute profiler to get detailed performance metrics and find out how the OpenACC is mapped at the Compute Unified Device Architecture(CUDA) hardware level. Note: You will get a better understanding of the GPU architecture in the CUDA notebooks.


To do this, let's use the [solution](../../source_code/openacc/SOLUTION/rdf_collapse.f90) as a reference to get a similar report from Nsight Compute. Run the application, and profile it with the Nsight Systems first.

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile the solution for Tesla GPU
!cd ../../source_code/openacc && nvfortran -acc -ta=tesla,lineinfo -Minfo=accel -o rdf nvtx.f90 SOLUTION/rdf_collapse.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

In [ ]:
#profile the solution with Nsight Systems 
!cd ../../source_code/openacc && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o rdf_collapse_solution ./rdf

Let's checkout the profiler's report. Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../../source_code/openacc/rdf_collapse_solution.qdrep) and open it via the GUI. Now, right click on the kernel `rdf_98_gpu` and click on "Analyze the Selected Kernel with NVIDIA Nsight Compute" (see below screenshot). 

<img src="../images/f_compute_analyz.png">

Then, make sure to tick the radio button next to "Display the command line to user NVIDIA Nsight Compute CLI". 

<img src="../images/compute_command_line.png" width="50%" height="50%">

Then, you simply copy the command, run it and analyze the selected kernel. 

<img src="../images/f_compute_command.png" width="50%" height="50%">

To profile the selected kernel, run the below cell (by adding `--set full` we make sure to capture all the sections in Nsight Compute profiler):

In [ ]:
#profile the selected kernel in the solution with Nsight compute
!cd ../../source_code/openacc && ncu --set full --launch-skip 1 --launch-count 1 -o rdf_collapse_solution ./rdf

Let's checkout the Nsight Compute profiler's report together. Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../../source_code/openacc/rdf_collapse_solution.ncu-rep) and open it via the GUI. Let's checkout the first section called "GPU Speed Of Light". This section gives an overview of the utilization for compute and memory resources on the GPU. As you can see from the below screenshot, the Speed of Light (SOL) reports the achieved percentage of utilization of 30.04% for SM and 70.10% for memory. 


<img src="../images/f_sol.png">


**Extra**: If you can use the baseline feature on the Nsight Compute and compare the analysis of the kernel from this version of the RDF (which uses data directives and collapse clause) with the very first parallel version where we only added parallel directives and used managed memory, you can see how much improvements we got (see the below screenshot for reference):

<img src="../images/f_sol_baseline.png">

It is clear that we were able to reduce the execution time to half(red rectangle) and increase the SM and memory utilization (green rectangle). However, as you see the device is still underutilized. Let's look at the roofline analysis which indicates that the application is bandwith bound and the kernel exhibits low compute throughput and memory is more heavily utilized than Compute and it is clear the the memory is the bottleneck.

<img src="../images/f_roofline_collapse.png">

The Nsight Compute profiler suggests us to checkout the "Memory Workload Analysis" report sections to see where the memory system bottleneck is. There are 9.85 M instructions loading from or storing to the global memory space. The link going from L1/TEX Cache to Global shows 8.47 M requests generated due to global load instructions.

<img src="../images/f_memory_collapse.png">


Let's have a look at the table showing L1/TEX Cache. The "Sectors/Req" column shows the average ratio of sectors to requests for the L1 cache. For the same number of active threads in a warp, smaller numbers imply a more efficient memory access pattern. For warps with 32 active threads, the optimal ratios per access size are: `32-bit: 4`, `64-bit: 8`, `128-bit: 16`. Smaller ratios indicate some degree of uniformity or overlapped loads within a cache line. Checkout the [GPU Architecture Terminologies](../GPU_Architecture_Terminologies.ipynb) notebook to learn more about threads and warps.

In the example screenshot, we can see that this number is higher. This implies uncoalesced memory accesses and will result in increased memory traffic. We are not efficiently utilizing the bytes transferred.

<img src="../images/f_memory_sec.png">

Now, let's have a look at the "Source Counters" section located at the end of "Details" page of the profiler report. The section contains tables indicating the N highest or lowest values of one or more metrics in the selected kernel source code. Hotspot tables point out performance problems in the source. 

<img src="../images/f_source_loc.png">

We can select the location links to navigate directly to this location in the "Source" page. Moreover, you can hover the mouse over a value to see which metrics contribute to it.

<img src="../images/f_source_hover.png">

The "Source" page displays metrics that can be correlated with source code. It is filtered to only show (SASS) functions that were executed in the kernel launch.

<!--<img src="../images/source_sass_collapse.png">-->

<img src="../images/f_source_sass.png">

The "Source" section in the "Details" page indicates  that the issue is *uncoalesced Global memory access*. 

<img src="../images/uncoalesced_hint.png">

**Memory Coalescing**

On GPUs, threads are executed in warps. When we have a group of 32 contiguous threads called *warp* accessing adjacent locations in memory, we have *Coalesced memory* access and as a result we have few transactions and higher utilization. However, if a warp of 32 threads accessing scattered memory locations, then we have *Uncoalesced  memory* access and this results in high number of transactions and low utilization.


<img src="../images/coalesced_mem.png">

Without changing the data structure and refactoring the code, we cannot fix this issue and improve the performance further using OpenACC in a straightforward easier way. The next step would be to look into how to optimize this application further with CUDA and perhaps take advantage of shared memory.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file [here] by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../nways_files.zip).
Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a></p>

-----


# Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).